In [1]:
import numpy as np

from grg_pheno_sim.multi_grg_phenotype import sim_phenotypes_multi_grg
from grg_pheno_sim.model import grg_causal_mutation_model


The following commands only serve the purpose of converting the VCF zip file into GRGs that will be used for phenotype simulation.

In [2]:
%%script bash --out /dev/null
if [ ! -f test-200-samples.grg ]; then
  grg construct -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples.grg
fi

In [3]:
%%script bash --out /dev/null
if [ ! -f test-200-samples_copy.grg ]; then
  grg construct -p 10 -t 2 ../data/test-200-samples.vcf.gz --out-file test-200-samples_copy.grg
fi

In [4]:
grg_list = ["test-200-samples.grg", "test-200-samples_copy.grg"]
#this is the list of GRG files to be loaded in 

We will first demonstrate loading all GRG files into RAM and simulating phenotypes. Causal mutations are sampled from each GRG, and the genetic values are obtained for the samples. The combined genetic dataframe is the addition of each GRG's genetic values (for each causal mutation ID). Noise is sampled at the end and added to obtain the phenotypes.

NOTE: It is necessary for each GRG to have the same number of samples.

In [5]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_phenotype = True #check for normalizing phenotyopes

normalize_genetic_values_before_noise = False

heritability = [0.33, 0.25]

load_all_into_RAM = True #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, normalize_phenotype=normalize_phenotype, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability)


Loaded test-200-samples.grg into RAM
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -19.937405                   0
1                0      -0.830400                   1
2                1     -23.747265                   0
3                1     -10.233794                   1
4                2     -24.899522                   0
..             ...            ...                 ...
395            197      10.774932                   1
396            198     -24.823388                   0
397            198     -13.646514                   1
398            199     -16.018854                   0
399            199     -12.833933                   1

[400 rows x 3 columns]
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -18.821014                   0
1                0      

In [6]:
multi_grg_multi_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,-0.003396,-0.484435,-0.487831
1,1,0,-0.191852,1.714743,1.522890
2,0,1,-0.084005,-0.080870,-0.164874
3,1,1,0.497242,1.577601,2.074842
4,0,2,-0.233244,-0.872150,-1.105394
...,...,...,...,...,...
395,1,197,0.533595,0.691440,1.225035
396,0,198,0.062669,0.776854,0.839523
397,1,198,0.806812,0.589597,1.396408
398,0,199,0.037010,-0.177205,-0.140196


We now perform similar simulations, but by loading the GRGs into RAM sequentially (instead of all together).

In [7]:
model_type = "multivariate normal"
means = np.zeros(2)
cov = np.array([[1, 0.25], [0.25, 1]])

model = grg_causal_mutation_model(model_type, mean=means, cov=cov)

num_causal_per_file = 1000

random_seed = 1

normalize_genetic_values_before_noise = False

heritability = [0.33, 0.25]

load_all_into_RAM = False #this parameter decides whether to load all GRGs into RAM together

multi_grg_multi_seq_phenotypes = sim_phenotypes_multi_grg(grg_list, model, num_causal_per_file, random_seed, load_all_ram=load_all_into_RAM, normalize_genetic_values_before_noise=normalize_genetic_values_before_noise, heritability=heritability)


Loaded test-200-samples.grg into RAM
Genetic values for test-200-samples.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0     -20.138065                   0
1                0     -29.834874                   1
2                1      14.599716                   0
3                1      18.241617                   1
4                2     -17.719872                   0
..             ...            ...                 ...
395            197       7.317286                   1
396            198      11.186771                   0
397            198      30.694741                   1
398            199      10.666451                   0
399            199      23.689830                   1

[400 rows x 3 columns]
Loaded test-200-samples_copy.grg into RAM
Genetic values for test-200-samples_copy.grg are as follows:
     individual_id  genetic_value  causal_mutation_id
0                0      22.958821                   0
1                0      

In [8]:
multi_grg_multi_seq_phenotypes

,causal_mutation_id,individual_id,genetic_value,environmental_noise,phenotype
0,0,0,2.820756,22.794766,25.615522
1,1,0,-34.209335,-22.860697,-57.070032
2,0,1,28.339329,-1.883971,26.455358
3,1,1,0.905370,6.198651,7.104020
4,0,2,16.937530,43.552362,60.489892
...,...,...,...,...,...
395,1,197,-14.094770,-7.277200,-21.371970
396,0,198,49.025824,-26.977058,22.048766
397,1,198,42.734347,24.667543,67.401890
398,0,199,28.830708,-18.541107,10.289602
